In [82]:
import os
import uproot
import ROOT as r
import math
import awkward as ak
import hist
from hist import Hist
import sys
import argparse
import mpl_plot_utilities as mplutils
import glob
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
#Loop over root files that contain analysis results
#indir = '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/test_run'
indir = '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103'
pattern = '*.root'

outdir = '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/summary_plots'
if not os.path.exists(outdir):
    os.mkdir(outdir)
pdf_all = PdfPages(f"{outdir}/zalpha_slope_analysis_summary_all_masses.pdf")
    
unsorted_root_files = glob.glob(f'{indir}/{pattern}')
root_files = sorted(unsorted_root_files, key=lambda x: float(os.path.basename(x).split('_')[1]))
histos_2d = {}

#loop over all mass files
for i,file_path in enumerate(root_files):
    print(file_path)
    mass = int(float(os.path.basename(file_path).split('_')[1]))
    slopes = []
    print(mass)
    
    with uproot.open(file_path) as file:
        for key in file.keys():
            if 'slope_' not in key:
                continue
            slope = key.split(';')[0].split('/')[0].replace('slope_','')
            slopes.append(float(slope))
        slopes = sorted(list(set(slopes)))[1:]
        print(slopes)
        

        #Make histograms for this mass
        bin_width = max(slopes)-min(slopes)/len(slopes)
        histos_2d[f'mass_{mass}_zbi'] = mplutils.defHist2d(f'mass_{mass}_zalpha_zbi', 100,0.5,100.5, len(slopes), 0+0.001, max(slopes)+0.001,xlabel='Iteration',ylabel='Slope')
        histos_2d[f'mass_{mass}_nsig'] = mplutils.defHist2d(f'mass_{mass}_zalpha_nsig', 100,0.5,100.5,len(slopes),0+0.001,max(slopes)+0.001, xlabel='Iteration',ylabel='Slope')
        histos_2d[f'mass_{mass}_nbkg'] = mplutils.defHist2d(f'mass_{mass}_zalpha_nbkg', 100,0.5,100.5,len(slopes),0+0.001,max(slopes)+0.001, xlabel='Iteration',ylabel='Slope')
    
        
        #Gather data
        max_nsig = 0
        max_nbkg = 0
        max_zbi = 0
        for slope in slopes:
            nbkg_values = file[f'slope_{slope}/nbkg_g'].values()
            nsig_values = file[f'slope_{slope}/nsig_g'].values()
            zbi_values = file[f'slope_{slope}/zbis_g'].values()
            if max(nbkg_values[1]) > max_nbkg:
                max_nbkg = max(nbkg_values[1])
            if max(nsig_values[1]) > max_nsig:
                max_nsig = max(nsig_values[1])
            if max(zbi_values[1]) > max_zbi:
                max_zbi = max(zbi_values[1])
            
            histos_2d[f'mass_{mass}_zbi'].fill(zbi_values[0], np.full(len(zbi_values[0]),slope), weight=zbi_values[1])
            histos_2d[f'mass_{mass}_nsig'].fill(nsig_values[0], np.full(len(nsig_values[0]),slope), weight=nsig_values[1])
            histos_2d[f'mass_{mass}_nbkg'].fill(nbkg_values[0], np.full(len(nbkg_values[0]),slope), weight=nbkg_values[1])
            
        zbi_fig,zbi_ax = mplutils.plotHist2d(histos_2d[f'mass_{mass}_zbi'],vmin=0.1, vmax=max_zbi,cmin=0.1, logZ=False, dpi=200)
        nsig_fig,nsig_ax = mplutils.plotHist2d(histos_2d[f'mass_{mass}_nsig'],vmin=0.5, vmax=max_nsig,cmin=0.5,logZ=False, dpi=200)
        nbkg_fig,nbkg_ax = mplutils.plotHist2d(histos_2d[f'mass_{mass}_nbkg'],vmin=0.5, vmax=max_nbkg,cmin=0.5,logZ=True, dpi=200)
        
        #save individual mass analysis results
        with PdfPages(f'{outdir}/mass_{int(mass)}_slope_summary.pdf') as pdf:
            pdf.savefig(zbi_fig)
            pdf.savefig(nsig_fig)
            pdf.savefig(nbkg_fig)
            
        #save all results to single pdf
        pdf_all.savefig(zbi_fig)
        pdf_all.savefig(nsig_fig)
        pdf_all.savefig(nbkg_fig)
        
            
        #Clear memory
        #plt.close('all')
            
pdf_all.close()

/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_40.0_zalpha_slope_study.root
40
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_55.0_zalpha_slope_study.root
55
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_60.0_zalpha_slope_study.root
60
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_65.0_zalpha_s

/sdf/group/hps/users/alspellm/projects/THESIS/analysis/plot_utilities/mpl_plot_utilities.py:87: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,ax = plt.subplots(dpi=dpi)


/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_85.0_zalpha_slope_study.root
85
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_90.0_zalpha_slope_study.root
90
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_95.0_zalpha_slope_study.root
95
[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
/sdf/group/hps/users/alspellm/projects/THESIS/analysis/tight_selection_studies/zalpha_small/run_batch/run_20230103/mass_100.0_zalpha_